In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
def tf_extractor(corpus):  
    # returns a frequency-based DTM
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1)) 
    features = vectorizer.fit_transform(corpus) # transform texts to a frequency matrix
    return vectorizer, features  

In [3]:
def tfidf_extractor(corpus):
    # returns a tf-idf based DTM
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [17]:
with open('/Users/soothingni/Downloads/NOUN_전체.txt', 'r') as f:
    total_docs = [x.strip().split(',') for x in f.readlines()]
    docs =  [(int(doc[3]), doc[4]) for doc in total_docs if len(doc) == 4]
    scores, texts = zip(*docs)

In [30]:
filtered_texts = []
filtered_labels = []

for score, text in zip(scores, texts):
    if 100 <= score <= 1000:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 0~100 -> 부정, 0
    # 1000~ -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 1000 else 0)

In [31]:
num_reviews = len(filtered_texts) #788189

num_train = int(num_reviews*0.7) #551732
# 전체 리뷰 중에서 70%를 training data로 사용하고, 나머지 30%를 test data로 사용
train_texts = filtered_texts[:num_train]
train_labels = filtered_labels[:num_train]
test_texts = filtered_texts[num_train+1:]
test_labels = filtered_labels[num_train+1:]

In [32]:
# You can also use the following method
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)

In [33]:
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
# input의 형태 = list of docs
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]
# tf_vectorizer.vocabulary_.items() returns a list of (word, frequency)
# We sort words based on their frequencies and save the words

In [34]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
# C = Inverse of regularization strength(==lamda), 즉 C 값이 작을수록 penalty를 많이 준다는 것입니다.
#C는 hyper parameter로, 학습을 통해 도출할 수 없음 / 사람이 지정해야함; 5가지 정도를 실행해보고 가장 정확한 결과를 도출하는 것을 활용
# penalty를 많이 준다는 뜻은 L1 같은 경우는 feature의 수를 그만큼 많이 줄인다는 뜻이고
# L2인 경우는 weight 값을 더 0에 가깝게 한다는 뜻입니다.
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 758 out of 30708
Accuracy: 0.98


In [35]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 756 out of 30708
Accuracy: 0.98


In [36]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
    
#기계학습 텍스트분석에서 각 단어는 하나의 변수; coefficient는 모델의 parameter에 해당된다; y = b1 + b0x에서 단어(tf or tfidf)는 x, coefficent는 b0임

[(52582, 3.900098221144166), (10963, 3.0996777281245906), (489, 2.934401698800163), (39051, 2.793706587186403), (35440, 2.4334745441335985)]
저희 (3.900)
내용 (3.100)
가해자 (2.934)
아이 (2.794)
수사 (2.433)
아들 (2.378)
사건 (2.123)
검찰 (1.988)
주민 (1.982)
대한 (1.920)
일본 (1.578)
청원 (1.565)
병원 (1.295)
일부 (0.619)
상황 (0.502)
가족 (0.274)
피해자 (0.258)
치료 (0.130)
처벌 (0.098)
가가 (0.000)
가가호호 (0.000)
가각 (0.000)
가감 (0.000)
가거대교 (0.000)
가거도 (0.000)
가건물 (0.000)
가게 (0.000)
가격 (0.000)
가격결정 (0.000)
가격담합 (0.000)
가격탄력성 (0.000)
가격표 (0.000)
가견 (0.000)
가결 (0.000)
가결의 (0.000)
가경 (0.000)
가경제 (0.000)
가계 (0.000)
가계도 (0.000)
가계부 (0.000)
가계부채 (0.000)
가계비 (0.000)
가계수표 (0.000)
가계약 (0.000)
가곘습까 (0.000)
가고시마 (0.000)
가고시마공항 (0.000)
가고일 (0.000)
가곡 (0.000)
가공 (0.000)
힘들곘습 (0.000)
힘들껍니 (0.000)
힘들닙다 (0.000)
힘들덴데 (0.000)
힘듧니 (0.000)
힘럾 (0.000)
힘를 (0.000)
힘마 (0.000)
힘모 (0.000)
힘보탭시 (0.000)
힘빠 (0.000)
힘살 (0.000)
힘셨으 (0.000)
힘쌔 (0.000)
힘쌔다 (0.000)
힘쌘 (0.000)
힘썻다 (0.000)
힘안 (0.000)
힘었는 (0.000)
힘을썻습니 (0.000)
힘이쌔 (0.000)
힘일 (0.000)
힘임 (0.000)
힘입